In [1]:
import uproot
import pandas as pd
import ROOT

Welcome to JupyROOT 6.22/02


In [2]:
DDks=uproot.open('/data3/tracking-upgrade-data/dump_comb/DD_ks_moore_big.root:Particle_Data')
DDl=uproot.open('/data3/tracking-upgrade-data/dump_comb/DD_l_moore_big.root:Particle_Data')
GDks=uproot.open('/data3/tracking-upgrade-data/dump_comb/GD_ks_moore_big.root:Particle_Data')
GDl=uproot.open('/data3/tracking-upgrade-data/dump_comb/GD_l_moore_big.root:Particle_Data')

In [3]:
DDvariables=['track_charge','track_chi2','track_chi2PerDoF','track_nLHCbIDs','track_p','track_phi','track_position_x',
           'track_position_y','track_position_r','track_pt','track_tx','track_ty','track_pseudoRapidity',
           'particle_hasScifi','particle_hasUT','particle_hasVelo','particle_isDown','particle_isDown_noVelo','particle_isLong',
           'particle_isLong_andUT','particle_isElectron']


GDvariables=['track_charge','track_chi2','track_chi2PerDoF','track_nLHCbIDs','track_p','track_phi','track_position_x',
           'track_position_y','track_position_r','track_pt','track_tx','track_ty','track_pseudoRapidity']



features=['track_chi2PerDoF','track_nLHCbIDs','track_p','track_position_x',
           'track_position_y','track_position_r','track_pt','track_tx','track_ty','track_pseudoRapidity']


tracks=['particle_hasScifi','particle_hasUT','particle_hasVelo','particle_isDown','particle_isDown_noVelo','particle_isLong',
           'particle_isLong_andUT','partiole_isElectron']

In [4]:
%%time

DDks_Data=DDks.arrays(DDvariables,library='pd')
DDl_Data=DDl.arrays(DDvariables,library='pd')


GDks_Data=GDks.arrays(GDvariables,library='pd')
GDl_Data=GDl.arrays(GDvariables,library='pd')



DD_data=pd.concat([DDks_Data,DDl_Data]).fillna(False)

GD_data=pd.concat([GDks_Data,GDl_Data]).fillna(False)


DD_data['Ghost']=0
GD_data['Ghost']=1

CPU times: user 2min 57s, sys: 7.65 s, total: 3min 5s
Wall time: 3min 4s


In [5]:
DD_data=DD_data.loc[(DD_data['track_p']<100e3) & (DD_data['particle_isDown_noVelo']==True) | (DD_data['particle_isLong_andUT']==True) &(DD_data['particle_isElectron']==False)]


GD_data=GD_data.loc[(GD_data['track_p']<100e3)]




In [6]:
DD_data=DD_data.sample(5000)
GD_data=GD_data.sample(5000)

In [7]:
compiled_data=pd.concat([DD_data,GD_data]).fillna(False)

In [8]:
#save

file=uproot.recreate('./compiled_sample.root')
file['Particle_Data']=compiled_data


#open
Data=ROOT.TFile('./compiled_sample.root')
tree=Data.Particle_Data

In [9]:
%jsroot on
c2=ROOT.TCanvas()
tree.SetLineColor(1)
tree.Draw('track_position_x','Ghost==0')
tree.SetLineColor(4)
tree.Draw('track_position_x','Ghost==1','SAME')

tree.SetLineColor(2)
tree.Draw('track_position_x','particle_isDown_noVelo==1','SAME')
c2.Draw()

In [10]:
DD_data.shape

(5000, 22)

In [11]:
GD_data.shape

(5000, 14)

In [12]:
2260518/(2260518+5643064)

0.28601183615226616

In [13]:
compiled_data.tail()

,track_charge,track_chi2,track_chi2PerDoF,track_nLHCbIDs,track_p,track_phi,track_position_x,track_position_y,track_position_r,track_pt,...,track_pseudoRapidity,particle_hasScifi,particle_hasUT,particle_hasVelo,particle_isDown,particle_isDown_noVelo,particle_isLong,particle_isLong_andUT,particle_isElectron,Ghost
639005,1.0,8.004656,0.667055,12.0,67041.119080,-0.231304,144.460892,-66.828316,159.169636,2161.760234,...,4.127270,False,False,False,False,False,False,False,False,1
745644,-1.0,5.219947,0.434996,12.0,4059.137017,-2.755600,-1239.621948,-1275.325684,1778.515722,1584.782729,...,1.593179,False,False,False,False,False,False,False,False,1
360967,-1.0,1.920426,0.174584,11.0,14940.314216,-2.839053,-282.139343,-191.782578,341.149771,1379.424433,...,3.073406,False,False,False,False,False,False,False,False,1
998186,-1.0,1.265916,0.105493,12.0,9490.801467,-2.942363,-531.673523,-244.581146,585.232152,1460.363715,...,2.558812,False,False,False,False,False,False,False,False,1
289695,-1.0,14.605761,1.217147,12.0,12781.701630,2.220871,146.399551,402.834595,428.612342,895.013242,...,3.350850,False,False,False,False,False,False,False,False,1


In [14]:
t=uproot.open('compiled_sample.root:Particle_Data')

In [15]:
p=t.arrays(library='pd')

In [16]:
p['Ghost'].value_counts()

0    5000
1    5000
Name: Ghost, dtype: int64